## Geo Political Agent 

Developed by Dr. Anish Roychowdhury 


This system implements a multi-agent AI architecture for analyzing commodity prices through
geopolitical intelligence gathering. It uses a orchestrated workflow of specialized agents to research,
analyze, and report on how global events impact commodity markets

### Prerequisites


**Required Dependencies**

- !pip install agents
- !pip install pydantic
- !pip install python-doten v
- !pip install asyncio
- !pip install IPython

### Imports 

In [ ]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import os
from typing import Dict, List, Optional, Union
from IPython.display import display, Markdown
from datetime import datetime
import json
import re
from enum import Enum

load_dotenv(override=True)

###  Data Models -Pydantic and Enum

The system uses structured data models to ensure consistent data flow:

### BaseModel (Pydantic)

- Purpose: Defines complex data structures with multiple fields
- Structure: Class with typed attributes and validation rules
- Usage: When you need objects with multiple properties and relationships
- Validation: Type checking, field validation, and data transformation
- Example: User profiles, analysis reports, complex configurations


### Enum (Enumeration)

- Purpose: Defines a fixed set of named constants/choices
- Structure: Simple key-value pairs with predefined options
- Usage: When you need to restrict values to specific choices
- Validation: Automatically validates that only defined values are used
- Example: Days of week, status codes, analysis types

### Data Models for Core Agentic Functions 


**GeopoliticalSearchItem**
- reason : Strategic justification for the search
- query : Specific search terms for geopolitical events
- priority : Impact level classification (high/medium/low)


**GeopoliticalSearchPlan**
- commodity : Target commodity for analysis
- searches : Collection of geopolitical search items

**CommoditySearchItem & CommoditySearchPlan**
- Similar structure focused on market-specific research

**CommodityAnalysisReport**
- executive_summary : High-level findings
- current_price_trend : Market direction analysis
- key_geopolitical_factors : Top impacting events
- price_drivers : Economic/political factors
- risk_assessment : Geopolitical risk evaluation
- market_outlook : Future predictions
- html_report : Formatted output document
- follow_up_analysis : Suggested next steps

In [2]:
class GeopoliticalSearchItem(BaseModel):
    reason: str = Field(description="Strategic reasoning for why this geopolitical search is critical for commodity price analysis.")
    query: str = Field(description="The specific search term focusing on geopolitical events, wars, policies, or world events.")
    priority: str = Field(description="Priority level: 'high', 'medium', or 'low' based on expected impact on commodity prices.")

class GeopoliticalSearchPlan(BaseModel):
    commodity: str = Field(description="The commodity being analyzed (oil, gold, copper, etc.)")
    searches: List[GeopoliticalSearchItem] = Field(description="List of geopolitical searches to understand global events affecting commodity prices.")

class CommoditySearchItem(BaseModel):
    reason: str = Field(description="Reasoning for this commodity-specific search query.")
    query: str = Field(description="Search term focused on commodity price trends, market analysis, or supply/demand factors.")

class CommoditySearchPlan(BaseModel):
    searches: List[CommoditySearchItem] = Field(description="List of commodity-specific searches for deeper market analysis.")

class CommodityAnalysisReport(BaseModel):
    executive_summary: str = Field(description="2-3 sentence executive summary of key findings and price trend predictions.")
    current_price_trend: str = Field(description="Current price trend analysis (bullish/bearish/neutral) with reasoning.")
    key_geopolitical_factors: List[str] = Field(description="Top 3-5 geopolitical factors currently impacting the commodity.")
    price_drivers: List[str] = Field(description="Primary economic and geopolitical drivers affecting price movements.")
    risk_assessment: str = Field(description="Assessment of geopolitical risks and their potential price impact.")
    market_outlook: str = Field(description="Short to medium-term market outlook and price predictions.")
    html_report: str = Field(description="Complete detailed report in HTML format with proper styling.")
    follow_up_analysis: List[str] = Field(description="Suggested areas for further geopolitical analysis.")




### Data Models for Natural Language Processing 

**QueryType (Enum)**

- PRICE_PREDICTION - Forecast future commodity price movements and trends
- GEOPOLITICAL_IMPACT - Analyze how political events affect commodity markets
- MARKET_ANALYSIS - Examine current market conditions, supply/demand dynamics
- RISK_ASSESSMENT - Identify and evaluate potential risks to commodity prices
- COMPARATIVE_ANALYSIS - Compare performance between multiple commodities
- GENERAL_COMMODITY - Broad market intelligence and overview analysis

In [3]:
class QueryType(Enum):
    PRICE_PREDICTION = "price_prediction"
    GEOPOLITICAL_IMPACT = "geopolitical_impact"
    MARKET_ANALYSIS = "market_analysis"
    RISK_ASSESSMENT = "risk_assessment"
    COMPARATIVE_ANALYSIS = "comparative_analysis"
    GENERAL_COMMODITY = "general_commodity"

**QueryIntent (BaseModel)**

- commodity - Primary commodity extracted from user query (e.g., "oil", "gold")
- query_type - Classification of analysis type requested (from QueryType enum)
- specific_focus - Key aspects to analyze (e.g., ["sanctions", "supply chain"])
- time_horizon - Analysis timeframe: short-term, medium-term, or long-term
- additional_commodities - Other commodities mentioned for comparison purposes
- confidence_score - AI confidence in query interpretation accuracy (0.0 to 1.0)

In [4]:
class QueryIntent(BaseModel):
    commodity: str = Field(description="Primary commodity identified in the query")
    query_type: QueryType = Field(description="Type of analysis requested")
    specific_focus: List[str] = Field(description="Specific aspects user wants analyzed")
    time_horizon: str = Field(description="Time frame for analysis (short-term, medium-term, long-term)")
    additional_commodities: List[str] = Field(default=[], description="Other commodities mentioned for comparison")
    confidence_score: float = Field(description="Confidence in query interpretation (0-1)")



**AnalysisScope (BaseModel)**

- primary_commodity - Main commodity for detailed analysis execution
- analysis_type - Specific analysis method to perform (QueryType)
- focus_areas - Priority topics to emphasize in research and reporting
- comparison_commodities - Secondary commodities for comparative analysis
- urgency_level - Processing priority: high, medium, or low urgency

In [5]:
class AnalysisScope(BaseModel):
    primary_commodity: str = Field(description="Main commodity to analyze")
    analysis_type: QueryType = Field(description="Type of analysis to perform")
    focus_areas: List[str] = Field(description="Specific areas to emphasize in analysis")
    comparison_commodities: List[str] = Field(default=[], description="Additional commodities for comparison")
    urgency_level: str = Field(description="Analysis urgency: high, medium, low")

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

###  Agent Definitions  for Core Functionality

**Geopolitical Planner Agent**

- `Purpose`: Strategic search planning for geopolitical events
- `Model`: GPT-4O-Mini
- `Input`: Commodity name
- `Output`: Structured search plan
- `Focus Areas`:
  - Military conflicts and tensions
  - Trade wars and sanctions
  - Central bank policies
  - Supply chain disruptions
  - Political instability in producing regions

In [6]:
# 1. GEOPOLITICAL PLANNER AGENT
GEOPOLITICAL_PLANNER_INSTRUCTIONS = """You are a senior geopolitical analyst specializing in commodity markets. 
Given a commodity (oil, gold, copper, etc.), create a strategic search plan to identify current geopolitical events, 
conflicts, policy changes, and world events that could impact the commodity's price.

Focus on:
- Active military conflicts and tensions
- Trade wars and sanctions
- Central bank policies and government decisions  
- Supply chain disruptions
- Political instability in key producing regions
- Environmental and regulatory changes
- International agreements and treaties

Generate 4-5 high-priority searches that will provide comprehensive geopolitical context."""

geopolitical_planner_agent = Agent(
    name="GeopoliticalPlannerAgent",
    instructions=GEOPOLITICAL_PLANNER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=GeopoliticalSearchPlan,
)

**Geopolitical Research Agent**
- `Purpose`: Execute geopolitical searches and analyze results
- `Model`: GPT-4O-Mini with required tool usage
- `Tools`: WebSearchTool (low context)
- `Output`: Factual, data-driven analysis (2-3 paragraphs)
- `Key Features`: Date-specific, location-aware, player-identified

In [7]:
 #GEOPOLITICAL RESEARCH AGENT  
GEOPOLITICAL_RESEARCH_INSTRUCTIONS = """You are a geopolitical research specialist focused on commodity markets.
Analyze web search results for geopolitical events and their potential impact on commodity prices.

Your analysis should be:
- Factual and data-driven
- Focused on price implications
- Concise but comprehensive (2-3 paragraphs max)
- Free of speculation, stick to reported facts
- Include specific dates, locations, and key players when available

Extract the essence of how geopolitical events translate to commodity market pressures."""

geopolitical_research_agent = Agent(
    name="GeopoliticalResearchAgent", 
    instructions=GEOPOLITICAL_RESEARCH_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)


**Commodity Market Planner Agent**

- `Purpose`: Plan commodity-specific market research
- `Model`: GPT-4O-Mini
- `Input`: Commodity + geopolitical context
- `Output`: Market-focused search plan
- `Focus Areas`:
  - Price movements and technical indicators
  - Supply/demand fundamentals
  - Market sentiment
  - Production data and inventory levels

In [8]:
# 3. COMMODITY MARKET PLANNER AGENT
COMMODITY_PLANNER_INSTRUCTIONS = """You are a commodity market analyst. Based on geopolitical context provided,
create targeted searches to understand current commodity price trends, market sentiment, supply/demand dynamics,
and technical analysis.

Focus on:
- Current price movements and technical indicators
- Supply and demand fundamentals  
- Market sentiment and trader positioning
- Production data and inventory levels
- Expert analyst predictions and forecasts
- Historical price patterns during similar geopolitical events

Generate 3-4 searches for deep commodity market analysis."""

commodity_planner_agent = Agent(
    name="CommodityPlannerAgent",
    instructions=COMMODITY_PLANNER_INSTRUCTIONS, 
    model="gpt-4o-mini",
    output_type=CommoditySearchPlan,
)

**Commodity Research Agent**

- `Purpose`: Execute market research and analysis
- `Model`: GPT-4O-Mini with required tool usage
- `Tools`: WebSearchTool (low context)
- `Output`: Quantitative market intelligence
- `Analysis Types`: Technical, fundamental, sentiment-based

In [9]:
COMMODITY_RESEARCH_INSTRUCTIONS = """You are a commodity market research specialist. 
Analyze web search results for commodity price data, market trends, and trading insights.

Your analysis should cover:
- Current price levels and recent movements
- Technical analysis and chart patterns
- Supply/demand fundamentals
- Market sentiment indicators
- Expert forecasts and price targets
- Trading volumes and institutional positioning

Keep analysis factual, quantitative when possible, and focused on actionable market intelligence.
Limit to 2-3 focused paragraphs per search."""

commodity_research_agent = Agent(
    name="CommodityResearchAgent",
    instructions=COMMODITY_RESEARCH_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini", 
    model_settings=ModelSettings(tool_choice="required"),
)

**Report Writer Agent**
- `Purpose`: Synthesize research into professional reports
- `Model`: GPT-4O-Mini
- `Input`: Combined geopolitical and market research
- `Output`: Structured HTML report with professional styling
- `Features`: Executive summaries, risk color-coding, data tables

In [10]:
REPORT_WRITER_INSTRUCTIONS = """You are a senior commodity analyst preparing reports for institutional clients.
Synthesize geopolitical research and commodity market analysis into a comprehensive, professional report.

Your report must include:
1. Executive Summary with clear price trend prediction
2. Current Market Analysis  
3. Geopolitical Risk Assessment
4. Key Price Drivers Analysis
5. Market Outlook and Recommendations
6. Risk Factors and Scenarios

Format the HTML report with:
- Professional styling with CSS
- Clear headings and sections
- Data tables where appropriate  
- Color coding for risk levels (green=low, yellow=medium, red=high)
- Executive summary highlighted at the top
- Footer with analysis date and disclaimer

Make it visually appealing and easy to scan for busy executives."""

report_writer_agent = Agent(
    name="ReportWriterAgent",
    instructions=REPORT_WRITER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=CommodityAnalysisReport,
)

**Output Agent**
- `Purpose`: File management and report persistence
- `Model`: GPT-4O-Mini
- `Tools`: Custom save_html_report function
- `Features`: Timestamp-based naming, error handling

### Define function tool for report generation for output agent 

In [11]:
# 6. OUTPUT AGENT
@function_tool
def save_html_report(filename: str, html_content: str) -> Dict[str, str]:
    """Save the HTML report to a file"""
    try:
        # Ensure reports directory exists
        os.makedirs("reports", exist_ok=True)
        
        # Create filename with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        full_filename = f"reports/{filename}_{timestamp}.html"
        
        with open(full_filename, 'w', encoding='utf-8') as f:
            f.write(html_content)
            
        return {
            "status": "success", 
            "filename": full_filename,
            "message": f"Report saved successfully to {full_filename}"
        }
    except Exception as e:
        return {
            "status": "error",
            "message": f"Failed to save report: {str(e)}"
        }

#### Define Output Agent 

In [12]:
OUTPUT_AGENT_INSTRUCTIONS = """You are responsible for saving analysis reports to HTML files.
You will receive a detailed commodity analysis report and should save it as an HTML file with an appropriate filename.

Use descriptive filenames that include the commodity name and analysis type.
Always confirm successful file creation."""

output_agent = Agent(
    name="OutputAgent",
    instructions=OUTPUT_AGENT_INSTRUCTIONS,
    tools=[save_html_report],
    model="gpt-4o-mini",
)

### Agent Definitions for Natural Language Processing

**Query Interpreter Agent**

- `Role:` Natural language query parser and intent extraction specialist
- `Input:` Raw user questions in natural language
- `Output:` Structured QueryIntent object with extracted information
- `Key Functions:`
  - Identifies primary commodity mentioned (oil, gold, wheat, etc.)
  - Classifies analysis type (price prediction, geopolitical impact, risk assessment)
  - Extracts specific focus areas (sanctions, supply disruption, weather effects)
  - Determines time horizon (short/medium/long-term)
  - Detects additional commodities for comparison
  - Provides confidence score for interpretation accuracy


`Example:` "What will oil prices do?" → {commodity: "oil", query_type: "price_prediction", confidence: 0.95}

In [13]:
QUERY_INTERPRETER_INSTRUCTIONS = """You are a natural language query interpreter specializing in commodity and geopolitical analysis requests.

Your task is to analyze user queries and extract:
1. Primary commodity mentioned (oil, gold, copper, wheat, etc.)
2. Type of analysis requested (price prediction, geopolitical impact, market analysis, etc.)
3. Specific focus areas (supply disruption, sanctions impact, weather effects, etc.)
4. Time horizon (short-term: days/weeks, medium-term: months, long-term: quarters/years)
5. Additional commodities for comparison
6. Confidence score in your interpretation

Common query patterns:
- "What will oil prices do given the current situation?" → price_prediction, oil, current geopolitical events
- "How are sanctions affecting gold markets?" → geopolitical_impact, gold, sanctions focus
- "Compare copper vs aluminum in the current market" → comparative_analysis, copper+aluminum
- "What are the risks to wheat supply?" → risk_assessment, wheat, supply chain focus

Be precise in commodity identification and analysis type classification."""

query_interpreter_agent = Agent(
    name="QueryInterpreterAgent",
    instructions=QUERY_INTERPRETER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=QueryIntent,
)

**Analysis Coordinator Agent**

- `Role:` Strategic research planner and resource allocation optimizer
- `Input:` QueryIntent object from interpreter
- `Output:` AnalysisScope object with execution plan
- `Key Functions:`
  - Determines which analysis components to activate (geopolitical, market, comparative)
  - Sets priority levels and depth for each analysis type
  - Identifies specific focus areas to emphasize in research
  - Decides if multiple commodity analysis is needed
  - Assigns urgency level for processing priority


Purpose: Creates optimal research strategy to answer user's specific question efficiently

In [14]:
ANALYSIS_COORDINATOR_INSTRUCTIONS = """You are an analysis coordinator that determines the optimal research strategy based on user query intent.

Based on the interpreted query, decide:
1. Which analysis components are needed (geopolitical, market, comparative)
2. Priority and depth of each analysis type
3. Specific focus areas to emphasize
4. Whether multiple commodities need analysis
5. Urgency level for the analysis

Create a structured analysis plan that will guide the research agents to provide exactly what the user is asking for."""

analysis_coordinator_agent = Agent(
    name="AnalysisCoordinatorAgent",
    instructions=ANALYSIS_COORDINATOR_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=AnalysisScope,
)


**Conversational Response Agent**

- `Role:` Technical-to-conversational translator and user experience specialist
- `Input:` Structured analysis reports from research agents
- `Output:` Natural language response in conversational format
- `Key Functions:`

 - Converts technical analysis into accessible language
 - Maintains professional accuracy while being engaging
 - Provides clear explanations for technical terms
 - Structures information in logical, digestible flow
 - Includes relevant caveats and limitations
 - Follows user-friendly response format (direct answer → supporting analysis → outlook)


- `Goal:` Bridge the gap between complex technical analysis and user-friendly communication

- `Workflow:` Query Interpreter → Analysis Coordinator → [Research Execution] → Conversational ResponseRetryClaude can make mistakes. Please double-check responses.

In [15]:
CONVERSATIONAL_RESPONSE_INSTRUCTIONS = """You are a conversational commodity analyst that presents technical analysis in an accessible, engaging way.

Transform structured analysis reports into natural language responses that:
1. Directly answer the user's original question
2. Use conversational tone while maintaining professional accuracy
3. Highlight key insights with clear explanations
4. Provide context for technical terms
5. Structure information in logical flow
6. Include relevant caveats and limitations

Response format:
- Start with direct answer to user's question
- Provide supporting analysis in digestible sections
- Use bullet points for key factors
- End with outlook and next steps
- Keep technical jargon minimal but precise"""

conversational_response_agent = Agent(
    name="ConversationalResponseAgent",
    instructions=CONVERSATIONAL_RESPONSE_INSTRUCTIONS,
    model="gpt-4o-mini",
)

### Define Core Orchestration Functions 

#### Geopolitical Analysis Functions

**`plan_geopolitical_searches(commodity: str)`**

- `Purpose`: Initiates strategic planning for geopolitical research queries
- `Input`: Single commodity name (e.g., "oil", "gold")
- `Process`: Uses geopolitical_planner_agent to create targeted search strategy
- `Output`: Returns GeopoliticalSearchPlan with 4-5 prioritized search items
- `Key Feature`: Converts commodity name into structured research plan focused on global event

In [16]:
# See note above about cost of WebSearchTool

async def plan_geopolitical_searches(commodity: str):
    """Use geopolitical planner to identify key geopolitical events to research"""
    print(f"🌍 Planning geopolitical searches for {commodity}...")
    result = await Runner.run(geopolitical_planner_agent, f"Commodity: {commodity}")
    print(f"📋 Planned {len(result.final_output.searches)} geopolitical searches")
    return result.final_output

**`execute_geopolitical_research(search_plan)`**

- `Purpose:` Concurrent execution of all geopolitical searches
- `Process:` Creates async tasks for each search item and runs them in parallel
- `Performance:` 4-5x faster than sequential execution
- `Output:` List of geopolitical analysis results with priority rankings

In [17]:
async def execute_geopolitical_research(search_plan: GeopoliticalSearchPlan):
    """Execute all geopolitical searches concurrently"""
    print("🔍 Executing geopolitical research...")
    tasks = [asyncio.create_task(geopolitical_search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("✅ Completed geopolitical research")
    return results

**`geopolitical_search(item)`**

- `Purpose:` Individual geopolitical event research and analysis
- `Process:` Executes web search using geopolitical research agent
- `Output:` Structured result with query, priority, and analysis content

In [18]:
async def geopolitical_search(item: GeopoliticalSearchItem):
    """Execute individual geopolitical search"""
    input_msg = f"Search Query: {item.query}\nAnalysis Focus: {item.reason}\nPriority: {item.priority}"
    result = await Runner.run(geopolitical_research_agent, input_msg)
    return {
        "query": item.query,
        "priority": item.priority,
        "analysis": result.final_output
    }

#### Commodity Market Analysis Functions

**`plan_commodity_searches(commodity, geopolitical_context)`**

- `Purpose:` Plan market research based on geopolitical insights
- `Input:` Commodity name + geopolitical analysis results
- `Process:` Uses commodity planner agent with geopolitical context
- `Output:` CommoditySearchPlan with targeted market research queries
- `Focus:` Price trends, supply/demand, technical indicators, forecasts

In [19]:
async def plan_commodity_searches(commodity: str, geopolitical_context: List[Dict]):
    """Plan commodity-specific searches based on geopolitical context"""
    print(f"📈 Planning commodity market searches for {commodity}...")
    context_summary = "\n".join([f"- {item['query']}: {item['analysis'][:200]}..." for item in geopolitical_context])
    input_msg = f"Commodity: {commodity}\nGeopolitical Context:\n{context_summary}"
    result = await Runner.run(commodity_planner_agent, input_msg)
    print(f"📊 Planned {len(result.final_output.searches)} commodity searches")
    return result.final_output

**`commodity_search(item)`**

- `Purpose:` Individual commodity market research execution
- `Process:` Uses commodity research agent for specific market analysis
- `Output:` Market analysis with query focus and detailed findings

In [20]:
async def commodity_search(item: CommoditySearchItem):
    """Execute individual commodity search"""
    input_msg = f"Search Query: {item.query}\nAnalysis Focus: {item.reason}"
    result = await Runner.run(commodity_research_agent, input_msg)
    return {
        "query": item.query, 
        "analysis": result.final_output
    }

**`execute_commodity_research(search_plan)`**

- `Purpose:` Parallel execution of commodity market research
- `Process:` Concurrent async execution of all commodity searches
- `Output:` Market intelligence results with price and trend analysis

In [21]:
async def execute_commodity_research(search_plan: CommoditySearchPlan):
    """Execute commodity market research"""
    print("📊 Executing commodity market research...")
    tasks = [asyncio.create_task(commodity_search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("✅ Completed commodity research")
    return results

#### Report Generation Functions 

**`generate_analysis_report(commodity, geo_results, commodity_results)`**

- `Purpose:` Synthesize all research into comprehensive analysis report
- `Input:` Commodity name + geopolitical results + market results
- `Process:` Uses report writer agent to create structured HTML report
- `Output:` CommodityAnalysisReport with executive summary, trends, risks, outlook

In [22]:
async def generate_analysis_report(commodity: str, geopolitical_results: List[Dict], commodity_results: List[Dict]):
    """Generate comprehensive analysis report"""
    print("📝 Generating comprehensive analysis report...")
    
    input_data = {
        "commodity": commodity,
        "geopolitical_analysis": geopolitical_results,
        "commodity_analysis": commodity_results,
        "analysis_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    
    input_msg = f"""
    Commodity: {commodity}
    Analysis Date: {input_data['analysis_date']}
    
    GEOPOLITICAL RESEARCH FINDINGS:
    {json.dumps(geopolitical_results, indent=2)}
    
    COMMODITY MARKET RESEARCH FINDINGS:  
    {json.dumps(commodity_results, indent=2)}
    
    Please synthesize this into a comprehensive commodity analysis report.
    """
    
    result = await Runner.run(report_writer_agent, input_msg)
    print("✅ Analysis report generated")
    return result.final_output


**`save_report(report, commodity)'**

- `Purpose:` Save analysis report to file system
- `Process:` Uses output agent to create timestamped HTML file
- `Output:` Saved report in /reports directory with success confirmation

In [23]:
async def save_report(report: CommodityAnalysisReport, commodity: str):
    """Save the analysis report using output agent"""
    print("💾 Saving analysis report...")
    filename = f"{commodity.lower()}_geopolitical_analysis"
    result = await Runner.run(output_agent, f"Filename: {filename}\nHTML Content: {report.html_report}")
    print("✅ Report saved successfully")
    return result



### Main  Orchestration Function 

**`analyze_commodity_geopolitics(commodity)`**

- `Purpose:` Complete end-to-end commodity analysis workflow

- **4-Phase Process:**
  - Phase 1: Geopolitical intelligence gathering
  - Phase 2: Commodity market analysis
  - Phase 3: Report synthesis & generation
  - Phase 4: File storage & output

- `Output:` Complete CommodityAnalysisReport with executive summary and trends
- `Status:` Original unchanged function - foundation of the entire system

In [24]:
async def analyze_commodity_geopolitics(commodity: str):
    """
    ORIGINAL MAIN FUNCTION - UNCHANGED
    Main function to execute the complete geopolitical commodity analysis workflow
    """
    
    with trace("Geopolitical Commodity Analysis"):
        print(f"🚀 Starting geopolitical analysis for {commodity.upper()}")
        print("=" * 60)
        
        # Phase 1: Geopolitical Analysis
        print("📍 PHASE 1: GEOPOLITICAL INTELLIGENCE GATHERING")
        geopolitical_plan = await plan_geopolitical_searches(commodity)
        geopolitical_results = await execute_geopolitical_research(geopolitical_plan)
        
        # Phase 2: Commodity Market Analysis  
        print("\n📍 PHASE 2: COMMODITY MARKET ANALYSIS")
        commodity_plan = await plan_commodity_searches(commodity, geopolitical_results)
        commodity_results = await execute_commodity_research(commodity_plan)
        
        # Phase 3: Report Generation
        print("\n📍 PHASE 3: REPORT SYNTHESIS & OUTPUT")
        analysis_report = await generate_analysis_report(commodity, geopolitical_results, commodity_results)
        
        # Phase 4: Save Report
        await save_report(analysis_report, commodity)
        
        print("\n" + "=" * 60)
        print("🎉 ANALYSIS COMPLETE!")
        print(f"📋 Executive Summary: {analysis_report.executive_summary}")
        print(f"📈 Price Trend: {analysis_report.current_price_trend}")
        
        return analysis_report

### Utility Functions for Natural Language Processing 

**COMMODITY_KEYWORDS**  Dictionary

Purpose: Comprehensive keyword mapping for commodity identification
Structure: 14 major commodities with associated terms and synonyms
Coverage:

Energy: Oil (crude, petroleum, WTI, Brent), Natural Gas (LNG, gas), Coal
Precious Metals: Gold (bullion, AU), Silver (AG)
Industrial Metals: Copper (CU), Aluminum (AL, bauxite), Iron Ore (steel, FE)
Agricultural: Wheat (grain, cereals), Corn (maize), Soybeans (soy, oilseeds), Coffee (arabica, robusta), Sugar (sucrose), Cotton (textile)


Usage: Enables flexible commodity detection from various user input formats



In [25]:
COMMODITY_KEYWORDS = {
    'oil': ['oil', 'crude', 'petroleum', 'wti', 'brent', 'energy'],
    'gold': ['gold', 'precious metals', 'bullion', 'au'],
    'silver': ['silver', 'ag', 'precious metals'],
    'copper': ['copper', 'cu', 'industrial metals'],
    'wheat': ['wheat', 'grain', 'cereals'],
    'corn': ['corn', 'maize', 'grain'],
    'natural_gas': ['natural gas', 'lng', 'gas', 'natgas'],
    'aluminum': ['aluminum', 'aluminium', 'al', 'bauxite'],
    'iron_ore': ['iron ore', 'iron', 'steel', 'fe'],
    'coal': ['coal', 'thermal coal', 'coking coal'],
    'coffee': ['coffee', 'arabica', 'robusta'],
    'sugar': ['sugar', 'sucrose'],
    'cotton': ['cotton', 'textile'],
    'soybeans': ['soybeans', 'soy', 'oilseeds'],
}

**`detect_commodities_in_text(text)`**

- `Purpose:` Intelligent commodity identification from natural language input

- `Process:`
  - Converts input text to lowercase for case-insensitive matching
  - Iterates through COMMODITY_KEYWORDS dictionary
  - Checks if any keyword appears in the user text
  - Returns list of detected commodities


- `Output:` List of commodity names found in text
- `Example:` "How will crude oil and gold perform?" → ["oil", "gold"]
- `Benefits:` Handles multiple commodities, synonyms, and informal terminology



In [26]:
def detect_commodities_in_text(text: str) -> List[str]:
    """Detect commodities mentioned in user text"""
    text_lower = text.lower()
    detected = []
    
    for commodity, keywords in COMMODITY_KEYWORDS.items():
        if any(keyword in text_lower for keyword in keywords):
            detected.append(commodity)
    
    return detected

**`extract_time_indicators(text)`**

- `Purpose:` Automatic time horizon detection from user queries

- `Time Categories:`
  - Short-term: today, tomorrow, this week, next week, immediate, now
  - Medium-term: this month, next month, quarter, coming months
  - Long-term: this year, next year, long term, future, outlook

- `Logic:` Prioritized matching (short → medium → long → default medium)
- `Output:` String indicating time horizon ("short-term", "medium-term", "long-term")
- `Default:` Returns "medium-term" if no indicators found
- `Example:` "What's the outlook for wheat?" → "long-term"

In [27]:
def extract_time_indicators(text: str) -> str:
    """Extract time horizon indicators from text"""
    text_lower = text.lower()
    
    short_term_indicators = ['today', 'tomorrow', 'this week', 'next week', 'short term', 'immediate', 'now']
    medium_term_indicators = ['this month', 'next month', 'quarter', 'medium term', 'coming months']
    long_term_indicators = ['this year', 'next year', 'long term', 'future', 'outlook']
    
    if any(indicator in text_lower for indicator in short_term_indicators):
        return "short-term"
    elif any(indicator in text_lower for indicator in medium_term_indicators):
        return "medium-term"
    elif any(indicator in text_lower for indicator in long_term_indicators):
        return "long-term"
    else:
        return "medium-term"  # default

### Natural Language Orchestration Functions 

#### Query Processing Functions

**`interpret_user_query(user_query)`**

- `Purpose:` Transform natural language into structured analysis intent
- `Process:`
  - Uses utility functions to detect commodities and time horizons
  - Provides context to Query Interpreter Agent
  - Extracts commodity, analysis type, focus areas, confidence score

- `Output:` QueryIntent object with structured interpretation
- `Intelligence:` Combines keyword detection with AI interpretation for accuracy






In [28]:
async def interpret_user_query(user_query: str) -> QueryIntent:
    """NEW FUNCTION: Interpret user's natural language query"""
    print("🧠 Interpreting user intent...")
    
    # Add basic commodity detection as context
    detected_commodities = detect_commodities_in_text(user_query)
    time_horizon = extract_time_indicators(user_query)
    
    context = f"""
    User Query: {user_query}
    
    Detected Commodities: {detected_commodities}
    Implied Time Horizon: {time_horizon}
    
    Please interpret this query and extract the analysis intent.
    """
    
    result = await Runner.run(query_interpreter_agent, context)
    print(f"✅ Query interpreted with {result.final_output.confidence_score:.2f} confidence")
    return result.final_output

**`plan_analysis_approach(query_intent, original_query)`**

- `Purpose:` Strategic planning for optimal analysis execution
- `Input:` QueryIntent object + original user query
- `Process:` Uses Analysis Coordinator Agent to create execution strategy
- `Output:` AnalysisScope object with detailed analysis plan
- `Decision` Points: Analysis components needed, priority levels, urgency, comparison requirements

In [29]:
async def plan_analysis_approach(query_intent: QueryIntent, original_query: str) -> AnalysisScope:
    """NEW FUNCTION: Plan the analysis approach based on query intent"""
    print("📋 Planning analysis approach...")
    
    input_msg = f"""
    Original Query: {original_query}
    
    Interpreted Intent:
    - Commodity: {query_intent.commodity}
    - Analysis Type: {query_intent.query_type.value}
    - Focus Areas: {query_intent.specific_focus}
    - Time Horizon: {query_intent.time_horizon}
    - Additional Commodities: {query_intent.additional_commodities}
    - Confidence: {query_intent.confidence_score}
    
    Create an optimal analysis plan to address this query.
    """
    
    result = await Runner.run(analysis_coordinator_agent, input_msg)
    print(f"📊 Analysis plan created for {result.final_output.primary_commodity}")
    return result.final_output

### Analysis Execution Functions

**`execute_targeted_analysis(analysis_scope)`**

- `Purpose:` Execute analysis based on planned scope
- `KEY INTEGRATION POINT`: Calls original analyze_commodity_geopolitics() function
- `Process:`
  - Runs primary commodity analysis using existing system
  - Handles comparative analysis for multiple commodities if needed
  - Manages concurrent execution with error handling

- `Output:` Dictionary with primary analysis and optional comparative results
- `Architecture:` Seamlessly integrates new NL layer with existing analysis pipeline

In [30]:
async def execute_targeted_analysis(analysis_scope: AnalysisScope) -> dict:
    """
    NEW FUNCTION: Execute analysis based on the planned scope
    KEY INTEGRATION POINT: This function calls the EXISTING analyze_commodity_geopolitics() function
    """
    print(f"🔬 Executing {analysis_scope.analysis_type.value} analysis...")
    
    results = {}
    
    # Execute primary commodity analysis using EXISTING FUNCTION
    print(f"📊 Running primary analysis for {analysis_scope.primary_commodity}...")
    primary_analysis = await analyze_commodity_geopolitics(analysis_scope.primary_commodity)
    results['primary_analysis'] = primary_analysis
    
    # Execute comparative analysis if needed
    if analysis_scope.comparison_commodities:
        print(f"📈 Running comparative analysis for {len(analysis_scope.comparison_commodities)} additional commodities...")
        comparison_tasks = [
            asyncio.create_task(analyze_commodity_geopolitics(commodity))
            for commodity in analysis_scope.comparison_commodities
        ]
        comparison_results = await asyncio.gather(*comparison_tasks, return_exceptions=True)
        
        results['comparative_analysis'] = {
            commodity: result 
            for commodity, result in zip(analysis_scope.comparison_commodities, comparison_results)
            if not isinstance(result, Exception)
        }
    
    print("✅ Targeted analysis completed")
    return results

**`generate_conversational_response(user_query, query_intent, analysis_results)`**

- `Purpose:` Convert technical analysis into user-friendly conversational format
- `Input:` Original query + intent + technical analysis results
- `Process:` Uses Conversational Response Agent to transform structured data
- `Output:` Natural language response that directly answers user's question
- `Features`: Maintains professional accuracy while being accessible and engaging

In [31]:
async def generate_conversational_response(user_query: str, query_intent: QueryIntent, analysis_results: dict) -> str:
    """NEW FUNCTION: Generate conversational response to user query"""
    print("💬 Generating conversational response...")
    
    # Prepare analysis summary for conversational agent
    primary_analysis = analysis_results['primary_analysis']
    
    input_msg = f"""
    User's Original Question: {user_query}
    
    Query Intent Analysis:
    - Commodity: {query_intent.commodity}
    - Analysis Type: {query_intent.query_type.value}
    - Focus Areas: {', '.join(query_intent.specific_focus)}
    - Time Horizon: {query_intent.time_horizon}
    
    Analysis Results:
    Executive Summary: {primary_analysis.executive_summary}
    Current Price Trend: {primary_analysis.current_price_trend}
    Key Geopolitical Factors: {primary_analysis.key_geopolitical_factors}
    Price Drivers: {primary_analysis.price_drivers}
    Risk Assessment: {primary_analysis.risk_assessment}
    Market Outlook: {primary_analysis.market_outlook}
    
    Please provide a conversational response that directly answers the user's question using this analysis.
    """
    
    result = await Runner.run(conversational_response_agent, input_msg)
    print("✅ Conversational response generated")
    return result.final_output




#### Main Natural Language Workflow

**`process_natural_language_query(user_query)`**

- `Purpose:` Master coordinator for complete natural language processing workflow
- `Input:` Raw user query in natural language

- **4-Phase Process:**
  - `Phase 1:` Query interpretation and intent extraction
  - `Phase 2:` Analysis planning and scope determination
  - `Phase 3:` Targeted analysis execution (calls original system)
  - `Phase 4:` Conversational response generation

- `Output:` Complete response package with original query, intent, analysis, and user-friendly answer
- `Integration:` Bridges natural language input to technical analysis output



In [32]:
async def process_natural_language_query(user_query: str) -> dict:
    """
    NEW FUNCTION: Process natural language query and route to appropriate analysis
    This function coordinates the natural language processing workflow
    """
    
    with trace("Natural Language Query Processing"):
        print(f"🎯 Processing query: '{user_query}'")
        print("=" * 60)
        
        # Phase 1: Interpret the query
        print("📍 PHASE 1: QUERY INTERPRETATION")
        query_intent = await interpret_user_query(user_query)
        print(f"🎯 Identified commodity: {query_intent.commodity}")
        print(f"🔍 Analysis type: {query_intent.query_type.value}")
        print(f"⚡ Confidence: {query_intent.confidence_score:.2f}")
        
        # Phase 2: Plan analysis approach
        print("\n📍 PHASE 2: ANALYSIS PLANNING")
        analysis_scope = await plan_analysis_approach(query_intent, user_query)
        print(f"📊 Primary focus: {analysis_scope.primary_commodity}")
        print(f"🎯 Analysis type: {analysis_scope.analysis_type.value}")
        
        # Phase 3: Execute targeted analysis
        print("\n📍 PHASE 3: TARGETED ANALYSIS EXECUTION")
        analysis_results = await execute_targeted_analysis(analysis_scope)
        
        # Phase 4: Generate conversational response
        print("\n📍 PHASE 4: RESPONSE GENERATION")
        conversational_response = await generate_conversational_response(
            user_query, query_intent, analysis_results
        )
        
        print("\n" + "=" * 60)
        print("🎉 QUERY PROCESSING COMPLETE!")
        
        return {
            "original_query": user_query,
            "query_intent": query_intent,
            "analysis_scope": analysis_scope,
            "technical_analysis": analysis_results,
            "conversational_response": conversational_response,
            "confidence_score": query_intent.confidence_score
        }

### Main Interface Functions 

**`chat_with_commodity_analyst(user_query)`**

- `Purpose:` Primary entry point for natural language commodity analysis
- `Role:` High-level interface that wraps the complete NL processing pipeline
- `Input`: User query string in natural language

- `Process:`
  - Calls process_natural_language_query() for complete analysis
  - Handles all error scenarios gracefully
  - Formats response for end-user consumption


- `Success Output:` Structured dictionary with:

  - User-friendly conversational answer
  - Confidence score for query interpretation
  - Analysis details (commodity, type, technical report)


- `Success status` and original query

  - Error Handling: Returns formatted error response with helpful message
  - Usage: Single function call for complete commodity analysis from natural language

In [33]:
async def chat_with_commodity_analyst(user_query: str) -> dict:
    """
    NEW MAIN INTERFACE: Natural language query interface
    This is the primary entry point for natural language queries
    """
    
    try:
        # Process the natural language query
        response = await process_natural_language_query(user_query)
        
        return {
            "success": True,
            "user_query": user_query,
            "answer": response["conversational_response"],
            "confidence": response["confidence_score"],
            "analysis_details": {
                "commodity": response["query_intent"].commodity,
                "analysis_type": response["query_intent"].query_type.value,
                "technical_report": response["technical_analysis"]["primary_analysis"]
            }
        }
        
    except Exception as e:
        return {
            "success": False,
            "user_query": user_query,
            "error": str(e),
            "answer": f"I apologize, but I encountered an error processing your query: {str(e)}. Please try rephrasing your question."
        }

**`interactive_commodity_chat()`**

- `Purpose:` Command-line interactive chat interface for continuous conversation
- `Interface Type:` Real-time conversational terminal application

- `Features:`
  - Welcome message with usage instructions
  - Continuous loop for multiple queries
  - Built-in help system with example queries
  - Graceful exit commands (quit, exit, q)
  - Input validation and error handling


- `User Experience:`

  - Emoji-enhanced interface for visual appeal
  - Analysis progress indicators ("🔍 Analyzing...")
  - Formatted responses with confidence scores
  - Analysis summary display (commodity | analysis type)


- `Error Resilience:` Handles keyboard interrupts and unexpected errors
- `Session Management:` Maintains conversation until user exits

In [34]:
async def interactive_commodity_chat():
    """NEW FUNCTION: Interactive chat interface for commodity analysis"""
    
    print("🎯 INTERACTIVE COMMODITY ANALYST")
    print("Ask me anything about commodity prices, geopolitical impacts, market trends!")
    print("Type 'quit' to exit, 'help' for examples")
    print("=" * 70)
    
    while True:
        try:
            user_input = input("\n💬 Your question: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("👋 Thank you for using the Commodity Analyst!")
                break
                
            elif user_input.lower() in ['help', 'h']:
                print("\n📋 Example queries:")
                print("• What will oil prices do given current tensions?")
                print("• How are sanctions affecting gold markets?")
                print("• Compare copper vs aluminum in current market")
                print("• What are risks to wheat supply this year?")
                print("• Will natural gas prices rise with winter?")
                continue
                
            elif not user_input:
                print("Please enter a question about commodities.")
                continue
            
            print("\n🔍 Analyzing your question...")
            
            response = await chat_with_commodity_analyst(user_input)
            
            if response["success"]:
                print(f"\n🤖 {response['answer']}")
                print(f"\n📊 Analysis: {response['analysis_details']['commodity']} | {response['analysis_details']['analysis_type']}")
                print(f"🎯 Confidence: {response['confidence']:.2f}")
            else:
                print(f"\n❌ {response['answer']}")
                
        except KeyboardInterrupt:
            print("\n👋 Chat interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ Unexpected error: {str(e)}")

### Unified Interface Class Summary 

#### CommodityAnalyst Class

- Purpose: Unified interface combining original programmatic and new natural language functionality
- Design: Clean wrapper class providing access to both analysis modes
- Initialization:

  - Sets system ready status
  - Displays startup confirmation messages
  - Initializes both geopolitical and NL processing systems



**Core Methods**

`analyze(commodity)`


- Purpose: Direct programmatic commodity analysis
- Process: Calls original analyze_commodity_geopolitics() function
- Output: Complete CommodityAnalysisReport object
- Usage: Traditional technical analysis interface

`chat(user_query)`


- Purpose: Natural language query processing
- Process: Calls chat_with_commodity_analyst() function
- Output: Conversational response dictionary with analysis details
- Usage: User-friendly natural language interface

`interactive_session()`


- Purpose: Launch interactive chat session
- Process: Calls interactive_commodity_chat() function
- Usage: Real-time conversational analysis experience

`batch_analysis(commodities)`


- Purpose: Multi-commodity analysis with error handling
- Process: Iterates through commodity list using original analysis function
- Features: Individual error handling, progress tracking, success/failure reporting
- Output: Dictionary with results for each commodity (success or error message)




In [35]:
#==================== UNIFIED INTERFACE CLASS (New) ====================

class CommodityAnalyst:
    """
    NEW UNIFIED INTERFACE: Combines both programmatic and natural language commodity analysis
    This class provides a clean interface to both the original and new functionality
    """
    
    def __init__(self):
        self.system_ready = True
        print("🎯 Commodity Analyst System Initialized")
        print("✅ Geopolitical analysis agents ready")
        print("✅ Natural language processing ready")
    
    async def analyze(self, commodity: str) -> CommodityAnalysisReport:
        """
        EXISTING FUNCTIONALITY: Direct commodity analysis
        Calls the original analyze_commodity_geopolitics() function
        """
        return await analyze_commodity_geopolitics(commodity)
    
    async def chat(self, user_query: str) -> dict:
        """
        NEW FUNCTIONALITY: Natural language query interface
        """
        return await chat_with_commodity_analyst(user_query)
    
    async def interactive_session(self):
        """NEW FUNCTIONALITY: Start interactive chat session"""
        await interactive_commodity_chat()
    
    async def batch_analysis(self, commodities: list) -> dict:
        """
        ENHANCED FUNCTIONALITY: Analyze multiple commodities
        Uses the original analyze_commodity_geopolitics() function for each commodity
        """
        results = {}
        for commodity in commodities:
            try:
                results[commodity] = await self.analyze(commodity)
                print(f"✅ Completed analysis for {commodity}")
            except Exception as e:
                results[commodity] = f"Error: {str(e)}"
                print(f"❌ Failed analysis for {commodity}: {e}")
        return results

### Production Interface Summary

`production_interface()`


- Purpose: Main menu-driven interface for Jupyter notebooks
- Features:

  - 6 menu options: Natural Language Query, Direct Analysis, Batch Analysis, Interactive Chat, Demo, Tests
  - Input validation and error handling
  - Single execution per choice (no infinite loop like original)
  - Formatted output with emojis and analysis details
  - Designed for one-time execution per cell

In [36]:
# ==================== JUPYTER-COMPATIBLE EXECUTION ====================

async def production_interface():
    """Jupyter-compatible production interface"""
    
    analyst = CommodityAnalyst()
    
    print("\n🎯 COMMODITY ANALYST - Choose Mode:")
    print("1. Natural Language Query")
    print("2. Direct Commodity Analysis") 
    print("3. Batch Analysis")
    print("4. Interactive Chat")
    print("5. Demo System")
    print("6. Run Tests")
    
    choice = input("\nSelect option (1-6): ").strip()
    
    try:
        if choice == "1":
            query = input("💬 Your question: ")
            response = await analyst.chat(query)
            print(f"\n🤖 {response.get('answer', 'Error processing query')}")
            if response.get('success'):
                print(f"📊 Analysis: {response['analysis_details']['commodity']} | {response['analysis_details']['analysis_type']}")
                print(f"🎯 Confidence: {response['confidence']:.2f}")
            
        elif choice == "2":
            commodity = input("📊 Commodity name: ")
            report = await analyst.analyze(commodity)
            print(f"\n📈 Executive Summary: {report.executive_summary}")
            print(f"📊 Current Trend: {report.current_price_trend}")
            print(f"⚠️  Risk Assessment: {report.risk_assessment}")
            
        elif choice == "3":
            commodities_input = input("📋 Commodities (comma-separated): ")
            commodities = [c.strip() for c in commodities_input.split(",")]
            results = await analyst.batch_analysis(commodities)
            for commodity, result in results.items():
                if isinstance(result, CommodityAnalysisReport):
                    print(f"✅ {commodity}: {result.executive_summary}")
                else:
                    print(f"❌ {commodity}: {result}")
                    
        elif choice == "4":
            print("\n🎯 INTERACTIVE CHAT MODE")
            print("Ask questions about commodities! Type 'quit' to exit.")
            print("=" * 50)
            
            while True:
                user_input = input("\n💬 Your question: ").strip()
                
                if user_input.lower() in ['quit', 'exit', 'q']:
                    print("👋 Exiting chat mode!")
                    break
                    
                elif user_input.lower() in ['help', 'h']:
                    print("\n📋 Example queries:")
                    print("• What will oil prices do given current tensions?")
                    print("• How are sanctions affecting gold markets?")
                    print("• Compare copper vs aluminum in current market")
                    print("• What are risks to wheat supply this year?")
                    continue
                    
                elif not user_input:
                    print("Please enter a question about commodities.")
                    continue
                
                print("\n🔍 Analyzing your question...")
                response = await analyst.chat(user_input)
                
                if response["success"]:
                    print(f"\n🤖 {response['answer']}")
                    print(f"📊 Analysis: {response['analysis_details']['commodity']} | {response['analysis_details']['analysis_type']}")
                    print(f"🎯 Confidence: {response['confidence']:.2f}")
                else:
                    print(f"\n❌ {response['answer']}")
                    
        elif choice == "5":
            print("\n🚀 Running System Demo...")
            await demo_integrated_system()
            
        elif choice == "6":
            print("\n🧪 Running Integration Tests...")
            await test_integration()
            
        else:
            print("Invalid choice. Please select 1-6.")
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")

# =


### Demo and Testing Functions

`quick_demo()`
- Purpose: Fast demonstration of both analysis modes
- Features:
  - Two-part demo: Natural Language + Direct Analysis
  - Pre-configured test queries (oil geopolitical tensions, gold analysis)
  - Truncated output for quick validation





In [37]:

async def quick_demo():
    """Quick demo of both analysis modes"""
    
    print("🎯 QUICK DEMO - Commodity Analysis System")
    print("=" * 50)
    
    analyst = CommodityAnalyst()
    
    # Demo 1: Natural Language Query
    print("\n1️⃣ Natural Language Query Demo:")
    query = "What will oil prices do with current geopolitical tensions?"
    print(f"💬 Query: {query}")
    
    response = await analyst.chat(query)
    if response["success"]:
        print(f"🤖 Response: {response['answer'][:200]}...")
        print(f"📊 Detected: {response['analysis_details']['commodity']}")
    else:
        print(f"❌ Error: {response['answer']}")
    
    # Demo 2: Direct Analysis
    print("\n2️⃣ Direct Analysis Demo:")
    print("📊 Analyzing gold commodity...")
    
    try:
        report = await analyst.analyze("gold")
        print(f"✅ Executive Summary: {report.executive_summary}")
        print(f"📈 Price Trend: {report.current_price_trend}")
    except Exception as e:
        print(f"❌ Error: {str(e)}")

`ask_commodity_question(question: str)`

- Purpose: Single-query interface for simple questions
- Features:

  - Takes natural language question as parameter
  - Returns formatted response with analysis details
  - Shows technical report key insights (trend, risk factors)
  - Most straightforward way to test the system
  - Ideal for programmatic usage or quick tests

In [38]:
# ==================== SIMPLE SINGLE QUERY FUNCTION ====================

async def ask_commodity_question(question: str):
    """Simple function to ask a single question"""
    
    print(f"💬 Processing: {question}")
    print("🔍 Analyzing...")
    
    analyst = CommodityAnalyst()
    response = await analyst.chat(question)
    
    if response["success"]:
        print(f"\n🤖 Answer: {response['answer']}")
        print(f"\n📊 Analysis Details:")
        print(f"   • Commodity: {response['analysis_details']['commodity']}")
        print(f"   • Analysis Type: {response['analysis_details']['analysis_type']}")
        print(f"   • Confidence: {response['confidence']:.2f}")
        
        # Show key insights from technical report
        tech_report = response['analysis_details']['technical_report']
        print(f"\n📈 Key Insights:")
        print(f"   • Current Trend: {tech_report.current_price_trend}")
        print(f"   • Top Risk Factors: {tech_report.key_geopolitical_factors[:2]}")
        
    else:
        print(f"\n❌ Error: {response['answer']}")
    
    return response

`test_integration()`

- Purpose: Systematic integration testing between old and new systems
- Test Strategy: Three-tier validation approach
- Error Handling: Comprehensive try-catch blocks with specific error reporting
- Output: Pass/fail status for each integration test with descriptive messages

**Test 1 - Original Function Validation:**

- Directly calls analyze_commodity_geopolitics() with gold commodity
- Validates CommodityAnalysisReport generation
- Confirms original functionality remains unchanged


**Test 2 - Natural Language Processing Validation:**

- Tests chat_with_commodity_analyst() with sample query
- Validates success status in response
- Confirms new NL functionality works independently


**Test 3 - Unified Interface Validation:**

- Tests CommodityAnalyst class with both analysis modes
- Validates direct analysis returns proper report object
- Confirms chat functionality returns successful response
- Ensures both interfaces work through unified class




In [39]:
async def test_integration():
    """NEW FUNCTION: Test integration between old and new systems"""
    
    print("🧪 INTEGRATION TEST")
    print("="*50)
    
    # Test 1: Original function still works
    print("Test 1: Original function...")
    try:
        report = await analyze_commodity_geopolitics("gold")
        print(f"✅ Original function works: {bool(report.executive_summary)}")
    except Exception as e:
        print(f"❌ Original function failed: {e}")
    
    # Test 2: Natural language processing works
    print("\nTest 2: Natural language processing...")
    try:
        response = await chat_with_commodity_analyst("What will gold prices do?")
        print(f"✅ Natural language works: {response['success']}")
    except Exception as e:
        print(f"❌ Natural language failed: {e}")
    
    # Test 3: Unified interface works
    print("\nTest 3: Unified interface...")
    try:
        analyst = CommodityAnalyst()
        direct_result = await analyst.analyze("copper")
        chat_result = await analyst.chat("How is copper performing?")
        print(f"✅ Unified interface works: {isinstance(direct_result, CommodityAnalysisReport) and chat_result['success']}")
    except Exception as e:
        print(f"❌ Unified interface failed: {e}")

### Main Driving Code 

TEST 1:   Ask question on a single commodity 

In [ ]:
await ask_commodity_question("What will oil prices do with current geopolitical tensions?")

TEST 2: Quick Demo 

In [41]:
#await quick_demo()

FULL Production Run 

In [42]:
#await production_interface()